In [2]:
## remember to run conn_cocolab from the terminal before running cells in this notebook!
import os
import urllib, cStringIO

import pymongo as pm

import matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import seaborn as sns
sns.set_context('poster')
sns.set_style('white')

import numpy as np
from __future__ import division
import scipy.stats as stats
import pandas as pd
import json
import re

from PIL import Image
import base64
import datetime

In [3]:
# directory & file hierarchy
iterationName = 'cdm_run_v3'
exp_path = 'museumstation'
analysis_dir = os.getcwd()
exp_dir = os.path.abspath(os.path.join(os.getcwd(),'../..','experiments'))
##
sketch_dir = os.path.join(analysis_dir,'sketches')
data_dir = os.path.join(analysis_dir,'usage_data')
if not os.path.exists(sketch_dir):
    os.makedirs(sketch_dir)
## dir where we can keep all sketches not binned by category    

In [ ]:
# set vars 
auth = pd.read_csv('auth.txt', header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'rxdhawkins.me' ## cocolab ip address

# have to fix this to be able to analyze from local
import pymongo as pm
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')
db = conn['kiddraw']
coll = db['cdm_run_v3']

In [13]:
data = pd.read_csv(os.path.join(data_dir, "scribble_annotate_ZC.csv"))

# change age from strings to integers
data['age'] = data['age'].str.replace('age','')
pd.to_numeric(data.age)

# transfer categorical columns into integers
data.category = pd.Categorical(data.category)
data['cat_code'] = data.category.cat.codes


select_cols = ['cat_code', 'age', 'num_strokes', 'scribble']
data = data.loc[:, select_cols]
print data.shape

(800, 4)


In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

train, test = train_test_split(data, test_size=0.2)

X_cols = ['cat_code', 'age', 'num_strokes']
train_x = train.loc[:, X_cols]
test_x = test.loc[:, X_cols]

# get scribble values
train_y = train.scribble
test_y = test.scribble

print train_x.shape, train_y.shape
print test_x.shape, test_y.shape

# train
logreg = LogisticRegression()
logreg.fit(train_x, train_y)
print logreg

test_pred = logreg.predict(test_x)
f1 = f1_score(test_y, test_pred, average='micro')
print f1

(640, 3) (640,)
(160, 3) (160,)
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
0.8125
